# 拉勾网数据分析师岗位分析

## 项目背景

学数据分析已经有几个月了，这里做一个完整的实战项目作为总结和查漏补缺。在郑州找工作的时候发现数据分析相关岗位特别少，就想着
分析一下全国岗位的分布，给自己的去处做出指导。

## 项目简介

因为要从后端 PHP 转行数据分析，这里就对两者进行简单的比较，着重分析了数据分析的地域分布，给自己找工作做出指导。

## 项目目的

用实际的数据给自己做出指导和解决一些疑惑，主要针对以下几个问题？  
1. 数据分析岗位的城市分布情况，哪些城市是好的去处？  
2. 数据分析岗位薪资分布情况是怎样的，和 PHP 相比较如何？  
3. 数据分析岗位对工作经验的要求是怎么样，以及不同工作经验下的薪酬分布；  
4. 上海市不同地区的岗位分布情况

## 数据来源

项目使用的数据，是用自己写的爬虫从拉钩网上爬取的。拉钩网的工作岗位数据比较规范和完整，非常适合用来分析工作。  

## 主要应用技术

1. 使用 python 编写爬虫，爬去所需数据  
2. 数据分析工具 pandas、可视化工具 pyecharts  

这里没用 matplotlib 做可视化是因为 matplotlib 用的比较多，相对比较熟练，这里用可视化新秀 pyecharts ，算是边用边学吧。

## 数据整理

### 导入所需要的包

In [108]:
import numpy as np
import pandas as pd
from pyecharts.charts import Bar,Map,Boxplot
from pyecharts import options as opts
import warnings
# 不显示 warning
warnings.filterwarnings('ignore')

### 读取和清理数据

读取爬虫爬取后的数据，存放在 excel 文件中的，看一下数据的结构。

In [127]:
data = pd.read_excel('lagou.xls')
# 随机看 2 条数据
data.sample(2)

,职位类型,城市,公司名,公司规模,福利待遇,工作地点,学历要求,工作类型,职位名称,薪资,工作年限,公司发展阶段,技能要求
501,数据分析,苏州,苏州瑞鹏信息技术有限公司,50-150人,NaN,工业园区,硕士,产品|需求|项目类,数据分析师,12k-24k,1-3年,不需要融资,SPSS数据分析SQL
680,数据分析,北京,雪球（北京）技术开发有限公司,150-500人,股票期权带薪年假年度旅游定期体检,朝阳区,本科,开发|测试|运维类,数据分析师-用户增长方向,18k-35k,3-5年,D轮及以上,数据分析


In [128]:
# 看一下数据健康状态
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
职位类型      1500 non-null object
城市        1500 non-null object
公司名       1500 non-null object
公司规模      1500 non-null object
福利待遇      1322 non-null object
工作地点      1498 non-null object
学历要求      1500 non-null object
工作类型      1500 non-null object
职位名称      1500 non-null object
薪资        1500 non-null object
工作年限      1500 non-null object
公司发展阶段    1500 non-null object
技能要求      1354 non-null object
dtypes: object(13)
memory usage: 152.4+ KB


这里可以看到共 1500 条数据，只有 `福利待遇、工作地点、技能要求` 这三列有缺失值，缺失也不严重。其中福利待遇和技能要求不做分析，可以忽略。工作地点一列在做城市中岗位地区分布的时候再进行处理。

#### 去除重复的行

In [129]:
# 去除重复的行
# 查看是否有重复的行
data.duplicated(['职位名称', '公司名', '工作年限']).sum()

127

根据 `'职位名称', '公司名', '工作年限'` 这三个查重复，我们看到是有重复行，重复的行视为公司重复发布，进行去除。

In [130]:
# 去除重复数据
new_data = data.drop_duplicates(['职位名称', '公司名', '工作年限'])

## 数据分析

### 地域性分析

In [137]:
# 获取数据分析的数据
analysis_data = new_data[new_data['职位类型'] == '数据分析']
city_group = analysis_data.groupby(['城市'])['职位类型'].count()
city_group.sum()

671

In [138]:
# 排序 截取前 15 名
city_group = city_group.sort_values(ascending=False)[:15]
# 处理数据的格式，默认 numpy.int64 格式 pyechart 无法正常显示，转换成 int
city_num_list = [int(i) for i in list(city_group.values)]

In [139]:
# 职位总需求量
city_list = list(city_group.index)
# 初始化显示的宽度
bar = Bar(init_opts = opts.InitOpts(width='700px'))
bar.add_xaxis(city_list)
bar.add_yaxis("数据分析需求分布", city_num_list)

bar.set_global_opts(
    yaxis_opts=opts.AxisOpts(name="数量"),
    xaxis_opts=opts.AxisOpts(name="城市"),
)
bar.render_notebook()

从分布结果来看北京最多，需求量全国第一，占总岗位量的 38% 左右。Top5 分别是：北京、上海、深圳、广州、杭州。而我的老家郑州岗位太过稀少。

从岗位的数量分布，我们可以看出：**工作机会主要分布在北上广深和杭州**，这些城市是我们将来更好的去处，拥有更多的机会。

### 总体薪酬情况

In [141]:
# 薪酬一列格式处理。首先对分割出数值区间，然后计算区间中间值。因为薪资区间有重叠，计算中间值方便分析。
def avg_salary(salary):
    s_list = salary.split('-')
    s_min = int(s_list[0][:-1])
    s_max = int(s_list[1][:-1])
    s_avg = float(s_min + s_max)/2
    return int(s_avg*1000)
new_data['avg_month_salary'] = new_data['薪资'].apply(avg_salary)

In [142]:
# 看一下最大值，最小值
new_data['avg_month_salary'].max(), new_data['avg_month_salary'].min()

(51500, 1000)

In [143]:
# 分割区间
bins = [i*5000 for i in range(int(new_data['avg_month_salary'].max()/5000)+2)]
labels = [i for i in range(int(new_data['avg_month_salary'].max()/5000)+1)]

In [144]:
# 进行切割
analysis_cuts = pd.cut(new_data[new_data['职位类型'] == '数据分析']['avg_month_salary'], bins, labels=labels)
php_cuts = pd.cut(new_data[new_data['职位类型'] == 'php']['avg_month_salary'], bins, labels=labels)

In [145]:
# 统计区间数量，并按照索引排序
analysis_cuts_num = pd.value_counts(analysis_cuts).sort_index()
analysis_cuts_num = [int(i) for i in list(analysis_cuts_num.values)]

php_cuts_num = pd.value_counts(php_cuts).sort_index()
php_cuts_num = [int(i) for i in list(php_cuts_num.values)]

In [146]:
# 初始化显示的宽度
bar = Bar(init_opts = opts.InitOpts(width='1000px'))
x_lables = [str(int(i/1000))+'-'+str(int((i+5000)/1000)) for i in bins]
x_lables.pop()
bar.add_xaxis(x_lables)

bar.add_yaxis("数据分析薪资分布", analysis_cuts_num)
bar.add_yaxis("PHP薪资分布", php_cuts_num)

bar.set_global_opts(
        yaxis_opts=opts.AxisOpts(name="数量"),
        xaxis_opts=opts.AxisOpts(name="薪资范围(单位 K)",
    ),
)
bar.render_notebook()

可以到 数据分析 和 PHP 的薪资分布都是右偏分布，随着薪资的增长，需求的人数也越来越少，这比较符合实情。都主要集中在 10k-20k 之间，
从薪资分布上来看，可以说数据分析丝毫不逊于 PHP 开发，但也无法证明转数据分析是一个明智的选择。这需要我们进行更多维度的比较。下面我们
看一下不同城市两种薪资分布情况。

### 不同城市薪酬分布情况

In [147]:
# 只分析 top5 的城市，其他城市数量太少，没有实际意义
top5_city_list = city_list[:5]
analysis_salary_data = new_data[new_data['职位类型'] == '数据分析'].groupby(['城市'])['avg_month_salary']
php_salary_data = new_data[new_data['职位类型'] == 'php'].groupby(['城市'])['avg_month_salary']

analysis_city_salary = []
php_city_salary = []
for city in top5_city_list:
    v1 = analysis_salary_data.get_group(city)
    v1 = [int(i) for i in list(v1)]
    analysis_city_salary.append(v1)
    
    v2 = php_salary_data.get_group(city)
    v2 = [int(i) for i in list(v2)]
    php_city_salary.append(v2)

In [148]:
c = Boxplot(init_opts = opts.InitOpts(height='500px'))
c.add_xaxis(top5_city_list)
c.add_yaxis("数据分析", c.prepare_data(analysis_city_salary))
c.add_yaxis("PHP", c.prepare_data(php_city_salary))
c.set_global_opts(title_opts=opts.TitleOpts(title="")).render_notebook()

这里我们取了 top5 的城市进行比较，可以看出 北京、杭州 的 PHP岗位 更具有优势，广州、深圳 的 数据分析岗位更具优势，而上海在薪资集中度上数据分析岗位是比PHP岗位薪资稍低的，但是看最大值是数据分析岗位是比PHP岗位更有优势的，也就是天花板相对较高。对于有长远来说数据分析更具有优势，天花板更高。

### 数据分析工作经验需求

In [149]:
# 把 应届毕业生、不限、1年以下，统一改成一年以下
for i in analysis_data['工作年限'].iteritems():
    if i[1] in ['应届毕业生', '不限']:
        analysis_data['工作年限'][i[0]] = '1年以下'

In [150]:
years_data = analysis_data[['城市', '工作年限']].groupby(['工作年限']).count()

In [151]:
# 按照年限进行排序
temp = {'1年以下':1, '1-3年':2, '3-5年':3, '5-10年':4, '10年以上':5}
years_data['sort'] = years_data.index.map(temp)
years_data = years_data.sort_values('sort')

In [153]:
# 初始化显示的宽度
bar = Bar(init_opts = opts.InitOpts(width='600px'))
x_lables = list(years_data.index)
bar.add_xaxis(x_lables)

bar.add_yaxis("数据分析工作分布", [int(i) for i in list(years_data['城市'].values)])

bar.set_global_opts(
        yaxis_opts=opts.AxisOpts(name="数量"),
        xaxis_opts=opts.AxisOpts(name="工作年限",
    ),
)
bar.render_notebook()

年限分布挺正常的，主要集中在 1-3年、3-5年，可以说这是市场上的主力军，同时可以看到 1年以下 需求量也还是可以的，对新人或转行的人员来讲还是比较友好的。

### 不同工作经验的薪酬分布

In [154]:
analysis_year_salary = new_data[new_data['职位类型'] == '数据分析']

In [155]:
year_salary = analysis_year_salary[['工作年限', 'avg_month_salary']].groupby(['工作年限'])['avg_month_salary']

In [156]:
year_list = ['1年以下', '1-3年', '3-5年', '5-10年', '10年以上']
year_salary_data = []
for year in year_list:
    v1 = year_salary.get_group(year)
    v1 = [int(i) for i in list(v1)]
    year_salary_data.append(v1)

In [157]:
c = Boxplot(init_opts = opts.InitOpts(height='500px'))
c.add_xaxis(year_list)
c.add_yaxis("年限-薪资分布", c.prepare_data(year_salary_data))
c.set_global_opts(title_opts=opts.TitleOpts(title="")).render_notebook()

可以看出，随着年龄的增长，整体薪资成上涨趋势，越干越值钱。

### 上海数据分析岗位地区分布

In [158]:
shanghai_area = analysis_year_salary[analysis_year_salary['城市'] == '上海']

In [159]:
shanghai_area = shanghai_area.groupby(['工作地点'])['工作地点'].count()

In [160]:
map = Map()
area_list = [list(i) for i in shanghai_area.iteritems()]

map.add("", area_list, 
        maptype="上海",#地图类型
   ).set_global_opts(
        title_opts=opts.TitleOpts(title="上海数据分析岗位分布图"),
        visualmap_opts=opts.VisualMapOpts(),
    )
map.render_notebook()

因为打算到上海，特意分析了以下上海岗位分布，可以看出浦东新区、徐汇区、杨浦区岗位相对较多，可以给自己租房位置提供一个指导建议。当然由于上海样本量比较少
，分析可能存在偏差，这个在将来单独爬取上海的数据进行分析。

## 分析结论

通过对岗位、薪资分布等进行分析比较，得出下面的结论：  
1. 数据分析岗位就业机会主要集中在北上广深这几个一线城市以及杭州，这些城市是较好的一些去处。  
2. 数据分析岗位和PHP岗位的薪资差别不大，都主要集中在 10k-20k，但数据分析的天花板更高。  
3. 数据分析岗位对工作经验的要求主要集中在 1-3年、3-5年，随着年限的增长工资也有明显的提升。同时，要求一年以下的数量也占 13% 左右，对新入行的相对友好。
4. 数据分析岗位在上海主要分布在浦东新区、徐汇区、杨浦区，租房的时候可以结合地铁线着重考虑这几个区的房源。  

## 思考和总结

该项目对目的主要是对数据分析岗位需求和薪资方面进行一个更直观的认知。同时由于是 PHP 转行数据分析，所以也想对两者进行一下比较，站在中立的态度上，数据分析在薪资和岗位上并不占什么优势，但这只是其中的一两个维度，其他的维度比方说加班情况、对生活的影响这些无法直接获取数据来进行比较，还有对未来的判断这些主观因素。综合自己来看，数据分析是一个挺棒的方向。  
这次分析熟练了整个分析流程，比较基础，主要是 pandas 的应用，由于想试一下可视化新秀 pyecharts，边用边学，由于版本的更新，网上的很多答案都是旧版本的，导致花费了一些时间，其他的还基本OK。